#* konlpy 커스터마이징
- 뉴스 분석에 쓰일 konlpy에 주식 종목명과 약어를 새로운 사용자 단어로 추가하기
- 구글클라우드에서는 새로운 단어가 추가 되지 않는 이상 매번 실행할 필요 없음
- 코랩에서 실행시 매번 실행 시켜야 함

In [ ]:
# 코랩 실행시 맨 처음 설치 필요
!pip install JPype1
!pip install konlpy
!pip install pyMySQL
!pip install -U finance-datareader

In [2]:
# import packages
import os
import numpy as np
import pandas as pd
import FinanceDataReader as fdr
import konlpy
from konlpy.tag import Okt
import pymysql
from os import replace
import requests
from tqdm.notebook import tqdm
from datetime import datetime
from sqlalchemy import create_engine
import ast

In [ ]:
# --konlpy 커스터마이징  ----------------------------------------------
# --------------------------------------------------------------------
# Konlpy에 한국거래소 상장종목 전체 종목명과 약명을 새로운 단어로 추가하기
# --------------------------------------------------------------------
# konlpy 디렉토리로 이동
os.chdir('konlpy/설치된/디렉토리/java')
# 현재 디렉토리 확인
os.getcwd()
# 압축 풀 임시 디렉토리 /temp 만들기
os.makedirs('./temp')
# 임시디렉토리로 이동
os.chdir('./temp')
# 압축 풀기
!jar xvf ../open-korean-text-2.1.0.jar

# FinanceDataReader로 한국거래소 상장종목 전체 종목을 가져오기
df_krx = fdr.StockListing('KRX')
# 종목명만 추출하기
name_list = df_krx['Name']
# 새로운 단어목록 생성을 위한 data 변수 선언
data = ''
# 종목명을 한줄씩(\n)씩 넣어서 목록 만들기
for one_name in name_list:
  data += one_name+'\n'
# 종목명 약명 새로운 단어로 넣기
# DB 연결 준비: 종목명 약어 읽어오기
conn = pymysql.connect(host= '호스트주소', port = 포트번호, user="아이디", password="패스워드", db="DB이름", charset = 'utf8')
# DB에서 종목병 약어 추출위한 sql 준비
sql = f"SELECT x.kor_name,x.kor_abb FROM DB이름.테이블이름 x"
# DB 검색결과를 dataframe에 저장
name_abb_df = pd.read_sql_query(sql, conn)
# 실행확인을 위한 화면 출력
print(name_abb_df.head(3))
# DB close
conn.close()
# 종목명 추가하기
name_list = name_abb_df['kor_name']
# 종목명을 한줄씩(\n)씩 넣어서 목록 만들기
for one_name in name_list:
  data += one_name+'\n'
# 종목명 약명 추가하기
name_list = name_abb_df['kor_abb']
# 종목명 약명을 한줄씩(\n)씩 넣어서 목록 만들기
for one_name in name_list:
  data += one_name+'\n'
# konlpy의 company_names.txt 사전으로 저장
with open("konlpy/설치된/디렉토리/java/temp/org/openkoreantext/processor/util/noun/company_names.txt", 'w') as f:
    f.write(data)
# 임시 작업 디렉토리로 이동 확인
os.chdir('konlpy/설치된/디렉토리/java/temp')
# 다시 압축
!jar cvf ../open-korean-text-2.1.0.jar *
# 코랩 실행시 Restart session 실행

# --konlpy 커스터마이징 END --------------------------------------------